In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder # <------------------ Esto es nuevo :)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer # <------------------ Esto es nuevo :)
from sklearn.model_selection import train_test_split, GridSearchCV

# Pipeline 
from sklearn.pipeline import Pipeline # <------------------ Esto es nuevo :)

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Métricas de evaluación
from sklearn.metrics import accuracy_score


# Para guardar el modelo
import pickle

In [19]:
df = pd.read_csv('./data/titanic_clean.csv')

In [20]:
# Definir preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Sex', 'Embarked']),  # OneHotEncoder para columnas categóricas
        ('age', QuantileTransformer(output_distribution='normal', n_quantiles=500), ['Age']),
        ('fare', QuantileTransformer(output_distribution='normal', n_quantiles=500), ['Fare'])
    ],
    remainder='passthrough'  # Mantener otras columnas sin cambios
)

In [21]:
# Definir los modelos y sus respectivos hiperparámetros para GridSearch
modelos = {
    'Regresión Logística': {
        'modelo': LogisticRegression(),
        'parametros': {
            'model__C': [0.01, 0.1, 1, 10, 100],
            'model__penalty': ['l1', 'l2'],
            'model__solver': ['liblinear', 'saga'],
            'model__max_iter': [100, 500, 1000]
        }
    },
    'Clasificador de Vectores de Soporte': {
        'modelo': SVC(),
        'parametros': {
            'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'model__C': [0.1, 1, 10]
        }
    },
    'Clasificador de Árbol de Decisión': {
        'modelo': DecisionTreeClassifier(),
        'parametros': {
            'model__splitter': ['best', 'random'],
            'model__max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador de Bosques Aleatorios': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3, 4],
            'model__max_features': ['sqrt', 'log2', None]
        }
    },
    'Clasificador de Gradient Boosting': {
        'modelo': GradientBoostingClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador AdaBoost': {
        'modelo': AdaBoostClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100]
        }
    },
    'Clasificador K-Nearest Neighbors': {
        'modelo': KNeighborsClassifier(),
        'parametros': {
            'model__n_neighbors': [3, 5, 7]
        }
    },
    'Clasificador XGBoost': {
        'modelo': XGBClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3]
        }
    },
    'Clasificador LGBM': {
        'modelo': LGBMClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3],
            'model__learning_rate': [0.1, 0.2, 0.3],
            'model__verbose': [-1]
        }
    },
    'GaussianNB': {
        'modelo': GaussianNB(),
        'parametros': {}
    },
    'Clasificador Naive Bayes': {
        'modelo': BernoulliNB(),
        'parametros': {
            'model__alpha': [0.1, 1.0, 10.0]
        }
    }
}

In [22]:
# División de datos
X = df.drop(['Survived'], axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [26]:
# Inicializar variables para almacenar los puntajes de los modelos y el mejor estimador
puntajes_modelos = []
mejor_precision = 0
mejor_estimador = None
mejor_modelo = None
estimadores = {}

In [31]:
# Iterar sobre cada modelo y sus hiperparámetros
for nombre, info_modelo in modelos.items():
    # Crear pipeline para el modelo
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('scaler', MinMaxScaler()),  # MinMaxScaler se aplica a todas las columnas después del preprocesamiento
        ('model', info_modelo['modelo'])  # Modelo placeholder
    ])
    
    # Inicializar GridSearchCV con el pipeline y los hiperparámetros
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=info_modelo['parametros'],
        cv=5,
        scoring='accuracy',
        verbose=0,
        n_jobs=-1,
    )

    # Ajustar GridSearchCV con los datos de entrenamiento
    grid_search.fit(X_train, y_train)
    
    # Hacer predicciones con el modelo ajustado
    y_pred = grid_search.predict(X_test)
    
    # Calcular la precisión de las predicciones
    precision = accuracy_score(y_test, y_pred)
    
    # Almacenar los resultados del modelo
    puntajes_modelos.append({
        'Modelo': nombre,
        'Precisión': precision
    })

    estimadores[nombre] = grid_search.best_estimator_
    
    # Actualizar el mejor modelo si la precisión actual es mayor que la mejor precisión encontrada
    if precision > mejor_precision:
        mejor_modelo = nombre
        mejor_precision = precision
        mejor_estimador = grid_search.best_estimator_

/home/carlos/aprendizaje_supervisado/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/home/carlos/aprendizaje_supervisado/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1160: UserWarning: Inconsistent values: penalty=l1 with l1_ratio=0.0. penalty is deprecated. Please use l1_ratio only.
  warnings.warn(
/home/carlos/aprendizaje_supervisado/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2

KeyboardInterrupt: 

In [32]:
# Convertir los resultados a un DataFrame para una mejor visualización
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)

# Imprimir el rendimiento de los modelos de clasificación
print("Rendimiento de los modelos de clasificación")
print(metricas.round(2))

# Imprimir el mejor modelo y su precisión
print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.2f}")

# Guardar el mejor modelo con pickle
with open('pipeline.pkl', 'wb') as archivo_estimador:
    pickle.dump(mejor_estimador, archivo_estimador)

Rendimiento de los modelos de clasificación
                                Modelo  Precisión
0                  Regresión Logística       0.81
1  Clasificador de Vectores de Soporte       0.81
2    Clasificador de Árbol de Decisión       0.80
3   Clasificador de Bosques Aleatorios       0.80
---------------------------------------------------
MEJOR MODELO DE CLASIFICACIÓN
Modelo: Regresión Logística
Precisión: 0.81
